In [1]:
!pip install pytreebank


!pip install google_trans_new

import random
import google_trans_new
from google_trans_new import google_translator  
# Import Library
import random
import torch, torchtext
from torchtext import data
import pandas as pd

import pytreebank

In [2]:

dataset = pytreebank.load_sst("/content/sentimentData")

for category in ["train", "test", "dev"]:
    with open("sst_{}.txt".format(category), "w") as outfile:
        for item in dataset[category]:
            outfile.write(
                "{}\t{}\n".format(
                    item.to_labeled_lines()[0][0] + 1, item.to_labeled_lines()[0][1]
                )
            )

In [3]:
df_train = pd.read_csv(r"/content/sst_train.txt",sep="  ",header=None)
df_test = pd.read_csv(r"/content/sst_test.txt",sep="  ",header=None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [4]:
df_train.columns  = ['text']
df_test.columns  = ['text']  

In [5]:
df_train['label'] = df_train['text'].apply(lambda x : x.split("\t")[0])
df_train['text'] = df_train['text'].apply(lambda x : x.split("\t")[1])

df_test['label'] = df_test['text'].apply(lambda x : x.split("\t")[0])
df_test['text'] = df_test['text'].apply(lambda x : x.split("\t")[1])


In [6]:
df_train.head()

,text,label
0,The Rock is destined to be the 21st Century 's...,4
1,The gorgeously elaborate continuation of `` Th...,5
2,Singer/composer Bryan Adams contributes a slew...,4
3,You 'd think by now America would have had eno...,3
4,Yet the act is still charming here .,4


#### Data Augmentation

In [7]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return " ".join([x for x in words])
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return " ".join([x for x in [random.choice(words)]])
    else:
        return " ".join([x for x in remaining])

#### Random Swap

In [8]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    
    sentence = " ".join([x for x in sentence])
    return sentence

#### Transaltion

In [9]:
### code for the transalation:::

def translation(sentence):
  translator = google_translator()

  available_langs = list(google_trans_new.LANGUAGES.keys()) 
  trans_lang = random.choice(available_langs) 
  translations = translator.translate(sentence, trans_lang) 
  translations_en_random = translator.translate(translations,'en') 

  return translations_en_random


In [10]:
## function to create only a threshold to pick up the data from the dataframe for the augmentaiton purposes:::::

def data_augmentation(data ,swapping=True,deletion=True,transalation=True ,threshold=0.3):

  tmp_df = data.sample(frac = threshold)

  if(swapping):
    tmp_swap = tmp_df.copy()
    tmp_swap['text'] = tmp_swap['text'].apply(lambda x : random_swap(x.split()))

    data = data.append(tmp_swap,ignore_index=True)

  if(deletion):
    tmp_del = tmp_df.copy()
    tmp_del['text'] = tmp_del['text'].apply(lambda x : random_deletion(x.split()))

    data = data.append(tmp_del,ignore_index=True)

  if(transalation):

    tmp_tran = tmp_df.copy()
    tmp_tran['text'] = tmp_tran['text'].apply(lambda x : translation(x))

    data = data.append(tmp_tran,ignore_index=True)

  
  return data


In [12]:
augmented_data = data_augmentation(df_train,deletion=True,transalation=False)
df_train.shape , augmented_data.shape
#augmented_data = pd.read_csv(r"augmented_data.csv")

((8544, 2), (13670, 2))

In [13]:
#augmented_data.to_csv("augmented_data.csv",index=False)

In [14]:
# Manual Seed
SEED = 43
torch.manual_seed(SEED)
text = data.Field(sequential=True,tokenize = 'spacy', batch_first =True, include_lengths=True)
label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [15]:
fields = [('text', text),('label',label)]

In [16]:
example = [data.Example.fromlist([augmented_data.text[i],augmented_data.label[i]], fields) for i in range(augmented_data.shape[0])] 

In [17]:
stanford_Data = data.Dataset(example, fields)

In [18]:
(train, valid) = stanford_Data.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))
(len(train), len(valid))

(11620, 2050)

In [19]:
vars(train.examples[35])

{'label': '3',
 'text': ['Eckstraordinarily', 'lame', 'and', 'Severely', 'boring', '.']}

In [20]:
text.build_vocab(train)
label.build_vocab(train)

In [21]:
print('Size of input vocab : ', len(text.vocab))
print('Size of label vocab : ', len(label.vocab))
print('Top 10 words appreared repeatedly :', list(text.vocab.freqs.most_common(10)))
print('Labels : ', label.vocab.stoi)

Size of input vocab :  16150
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 9920), (',', 8862), ('the', 7486), ('and', 5513), ('a', 5468), ('of', 5356), ('to', 3769), ('-', 3336), ('is', 3161), ("'s", 3127)]
Labels :  defaultdict(<function _default_unk_index at 0x7f7fdb5d49d8>, {'4': 0, '2': 1, '3': 2, '5': 3, '1': 4})


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.text),
                                                            sort_within_batch=True, device = device)

In [23]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(text.vocab.stoi, tokens)

In [24]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [25]:
# Define hyperparameters
size_of_vocab = len(text.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 3
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [26]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(16150, 300)
  (encoder): LSTM(300, 100, num_layers=3, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)
The model has 5,167,905 trainable parameters


In [27]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [28]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        text, text_lengths = batch.text   
        
        # convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [29]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            text, text_lengths = batch.text
            
            # convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [30]:
N_EPOCHS = 100
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.584 | Train Acc: 26.89%
	 Val. Loss: 1.583 |  Val. Acc: 27.12% 

	Train Loss: 1.563 | Train Acc: 32.67%
	 Val. Loss: 1.570 |  Val. Acc: 30.29% 

	Train Loss: 1.517 | Train Acc: 37.86%
	 Val. Loss: 1.536 |  Val. Acc: 34.76% 

	Train Loss: 1.457 | Train Acc: 44.67%
	 Val. Loss: 1.517 |  Val. Acc: 37.64% 

	Train Loss: 1.405 | Train Acc: 50.62%
	 Val. Loss: 1.503 |  Val. Acc: 38.80% 

	Train Loss: 1.355 | Train Acc: 56.04%
	 Val. Loss: 1.494 |  Val. Acc: 39.86% 

	Train Loss: 1.313 | Train Acc: 60.66%
	 Val. Loss: 1.483 |  Val. Acc: 41.20% 

	Train Loss: 1.274 | Train Acc: 65.03%
	 Val. Loss: 1.480 |  Val. Acc: 41.06% 

	Train Loss: 1.239 | Train Acc: 68.48%
	 Val. Loss: 1.468 |  Val. Acc: 42.84% 

	Train Loss: 1.208 | Train Acc: 71.45%
	 Val. Loss: 1.465 |  Val. Acc: 43.08% 

	Train Loss: 1.182 | Train Acc: 73.87%
	 Val. Loss: 1.454 |  Val. Acc: 44.71% 

	Train Loss: 1.157 | Train Acc: 76.49%
	 Val. Loss: 1.448 |  Val. Acc: 45.53% 

	Train Loss: 1.139 | Train Acc: 77.91%
	

In [30]:
CUDA_LAUNCH_BLOCKING=1